In [1]:
from tvplayer import TVPlayer
import time

In [2]:
player = TVPlayer(["nbc","abc"])
playlist_file, schedules,info=player.initialize()

In [3]:
print(info)

{0: {'run_time': 2723.0818990000002, 'channel_sections': [{'playlist_index': 0, 'start_time_abs': 0, 'start_time_rel': 0, 'stop_time_abs': 480, 'stop_time_rel': 480, 'video_length': 2595.358}, {'playlist_index': 1, 'start_time_abs': 480, 'start_time_rel': 0, 'stop_time_abs': 490.005, 'stop_time_rel': 10.005, 'video_length': 10.005}, {'playlist_index': 2, 'start_time_abs': 490.005, 'start_time_rel': 0, 'stop_time_abs': 507.98963299999997, 'stop_time_rel': 17.984633, 'video_length': 17.984633}, {'playlist_index': 3, 'start_time_abs': 507.98963299999997, 'start_time_rel': 480, 'stop_time_abs': 987.9896329999999, 'stop_time_rel': 960, 'video_length': 2595.358}, {'playlist_index': 4, 'start_time_abs': 987.9896329999999, 'start_time_rel': 0, 'stop_time_abs': 1005.974266, 'stop_time_rel': 17.984633, 'video_length': 17.984633}, {'playlist_index': 5, 'start_time_abs': 1005.974266, 'start_time_rel': 0, 'stop_time_abs': 1023.8942659999999, 'stop_time_rel': 17.92, 'video_length': 17.92}, {'playlis

In [4]:
player.start()

started


In [3]:
player.start()
time.sleep(10)
print("changing channel")
player.change_channel(1)
time.sleep(10)
print("changing channel")
player.change_channel(0)
time.sleep(10)
print("changing channel")
player.change_channel(1)
time.sleep(10)
print("shutdown")
player.stop()


started
changing channel
Offset:10.00442886352539
found index at:0
seeking:10.00442886352539
chosen index:13
changing channel
Offset:20.26692795753479
found index at:0
seeking:20.26692795753479
chosen index:0
changing channel
Offset:30.527865409851074
found index at:0
seeking:30.527865409851074
chosen index:13
shutdown


In [ ]:
player.channel_up()

In [ ]:
player.stop()

In [40]:
for i in range(0,1):
    print(i)

0


In [83]:
test=[{},{},{}]
for e in test:
    e["test"]=1
print(test)

[{'test': 1}, {'test': 1}, {'test': 1}]


In [ ]:
with Telnet('127.0.0.1',1234) as tn:
    tn.write('1.mp4\n\c'.encode('utf-8'))

In [125]:
test=[1,2,3]
print(test.last())

AttributeError: 'list' object has no attribute 'last'